# **Implémentation de l'algorithme DQN à partir de l'observation de la RAM dans le jeu Pong :**

On commence par installer les dépendances suivantes :

In [ ]:
!sudo apt-get update
!sudo apt-get install -y xvfb ffmpeg
!pip install 'imageio==2.4.0'
!pip install pyvirtualdisplay
!pip install tf-agents

In [ ]:
from __future__ import absolute_import, division, print_function

import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import pyvirtualdisplay
import gym
import tf_agents
import random

import tensorflow as tf

from tf_agents.networks import q_network
from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import suite_gym, suite_atari
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import sequential
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory
from tf_agents.specs import tensor_spec
from tf_agents.utils import common
from tf_agents.environments import py_environment

*Création de notre environnement Python :*

Avec des outils comme gym, on peut définir plusieurs environnements à charger et cela sur différents jeux. C'est ce que nous avons fait pour le jeu qui nous intéresse, le jeu 'Pong'.
Lorsque l'on défini l'environnement à utiliser on peut chosir d'observer les images ou la RAM. Dans cette version, c'est la seconde option qui a été choisie.
Cependant, toutes les valeurs ne nous intéressent pas. On réduit alors au nombre de 5 les valeurs auxquelles nous nous intéressons.
Il y a la position de la raquette de notre joueur, la position de la balle selon deux coordonnées x et y, et sa vélocité selon toujours les deux axes x et y.

In [ ]:
# Notre environnement  
class NewEnv(py_environment.PyEnvironment) :
  def __init__(self,jeu) :
    # on va modifier un environnement déjà existant 
    self.jeu = jeu 
    # on modifie seulement les observations afin d'avoir un tableau avec 5 éléments, ces deniers sont des flottants
    self.obs_spec = tf_agents.specs.TensorSpec(shape=(5,),dtype=np.float32,name='observation')
    self.time = tf_agents.specs.ArraySpec(shape = (5,), dtype=np.float32,name='observation')
    

  def ram_val(self):
        
        # une liste pour stocker nos valeurs
        obs = []
        # on récupère la RAM correspondante à l'observation à l'état courant 
        state = self.jeu.current_time_step().observation
        # on convertit cette dernière en liste pour récupérer des valeurs
        ram = state.tolist()

        cpu_score = ram[13]      # le score de l'adversaire
        player_score = ram[14]   # notre score       
                
        player_paddle = ram[60]    # coordonnée ne notre joueur selon l'axe y 
        ball_x = ram[49]           # position selon x de la balle 
        ball_y = ram[54]           # position selon y de la balle

        velocity_x = ram[58] # vélocité de la balle selon l'axe x
        velocity_y = ram[56] # vélocité de la balle selon l'axe y

        # on ajoute dans une liste les valeurs dans la RAM qui nous intéressent
        obs.append(player_paddle)
        obs.append(ball_x)
        obs.append(ball_y)
        obs.append(velocity_x)
        obs.append(velocity_y)
      
        # on convertit la liste en tableau
        obs = np.array(obs, dtype=np.float32)

        # on retourne le tableau 
        return obs
    
  """
  On doit définir les différentes fonctions pour un environnement python
  """

  # la foncion step(a)
  # on modifie uniquement la partie observation de notre environnement
  def _step(self,a):
        b = self.jeu.step(a)
        c = tf_agents.trajectories.time_step.TimeStep(b.step_type, b.reward, b.discount, self.ram_val())
        return c

  # on modifie uniquement l'observation encore une fois
  def current_time_step(self):
    a = self.jeu.current_time_step()
    b = tf_agents.trajectories.time_step.TimeStep(a.step_type, a.reward, a.discount, self.ram_val())
    return b

  # les spécificités(forme, type de valeurs) des observations changent, on passe d'un tableau à 128 éléments à un tableau avec 5 éléments
  def observation_spec(self):
    return self.obs_spec

  # on a modifié les spécifications de l'observation, on applique alors ces changements pour les états
  def time_step_spec(self):
    a = self.jeu.time_step_spec()
    b = tf_agents.trajectories.time_step.TimeStep(a.step_type, a.reward, a.discount, self.time)
    return b

  # on reset notre environnement et on retourne état courant
  def _reset(self):
    self.jeu.reset()
    obs = self.current_time_step()
    return obs

  # les actions ne sont pas modifiées
  def action_spec(self):
    a = self.jeu.action_spec()
    return a


Différentes fonctions utilisées : 

In [ ]:
# Fonction pour évaluer une stratégie : le rendement moyen
def compute_avg_return(environment, policy, num_episodes=10):

  total_return = 0.0
  # le nombre de parties ou épisodes joué est de 10 
  for _ in range(num_episodes):
    time_step = environment.reset()
    episode_return = 0.0
    # tant que l'état ne correspond pas à la dernière étape d'une partie (une partie se termine quand un des joueurs arrive à 21 points) :
    while not time_step.is_last():
      # on choisit une action à partir de la stratégie définie en paramètre
      action_step = policy.action(time_step)
      # on obtient une observation/step/étape/état à partir de l'action jouée
      time_step = environment.step(action_step.action)
      # on ajoute la récompénse obtenue 
      episode_return += time_step.reward
    total_return += episode_return

  # on divise le total obtenu par le nombre d'épisodes que l'on a joué afin d'obtenir le rendement moyen
  avg_return = total_return / num_episodes
  return avg_return.numpy()[0]

"""
Collecte de données
"""

def collect_step(environment, policy, buffer):
  time_step = environment.current_time_step()
  action_step = policy.action(time_step)
  next_time_step = environment.step(action_step.action)
  traj = trajectory.from_transition(time_step, action_step, next_time_step)

  # La trajectory est ajouté dans le replay buffer
  # Celle-ci contient les spécifications des observations, actions, récompenses récupérés précédemment
  buffer.add_batch(traj)

# On collecte un nombre de données en fonction d'un nombre d'étapes qui est un hyperparamètre de l'algorithme
def collect_data(env, policy, buffer, steps):
  for _ in range(steps):
    collect_step(env, policy, buffer)

"""
Pour permettre la visualisation d'une partie
"""

def embed_mp4(filename):
  """Embeds an mp4 file in the notebook."""
  video = open(filename,'rb').read()
  b64 = base64.b64encode(video)
  tag = '''
  <video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
  Your browser does not support the video tag.
  </video>'''.format(b64.decode())

  return IPython.display.HTML(tag)


In [ ]:
# Met en place un affichage virtuel pour les environnements (cette cellule ne doit être éxécuté qu'une seule fois)
display = pyvirtualdisplay.Display(visible=0, size=(1400, 900)).start()

*Hyperparamètres et paramètres de l'agent :*

In [ ]:
"""
Hyperparamètres
"""
# le nombre d'itérations correspond au nombre de steps/étapes/états/observations parcourus
num_iterations = 500000
# le nombre de données collectées lors de l'initialisation, l'entraînement ne pouvant pas commencer avec aucune données.
initial_collect_steps = 200  
# nombre de données collectées à chaque itération 
collect_steps_per_iteration = 5
# taille du replay buffer
replay_buffer_max_length = 100000

# nombre de steps avant la mise à jour du réseau
batch_size = 32
# taux d'apprentissage pour l'optimizer
learning_rate = 2.5e-3

# affichage de la perte tous les 2500 steps
log_interval = 2500

# 5 épisodes soit 5 parties jouées, une partie s'arrête lorque un des joueurs arrive à 21
num_eval_episodes = 5  

# calcul du rendement moyen tous les 5000 steps
eval_interval = 5000  

"""
Paramètres de l'agent
"""
# Probabilité de choisir une action aléatoire
epsilon_greedy = 0.1
# Nombre de steps à considérer lors des calculs de TD erro et TD Loss
n_step_update=1
# Coefficient des mises à jour 'soft' du réseau
target_update_tau=1
# Période pour la mise à jour 'soft' du réseau
target_update_period=5
# Coefficient d'actualisation pour les récompenses
gamma=0.99
# Facteur multiplicatif pour les récompenses
reward_scale_factor=1.0


In [ ]:
"""
Les différents environnements de Pong dans gym

On peut ajouter '-ram' pour observer la ram au lieu d'images.
On peut mettre 'Deterministic' suivi de '-v4' afin de skipper 4 images à chaque step

"""
# Le nom de notre environnement
env_name = 'Pong-ramDeterministic-v4'

env = suite_gym.load(env_name)

"""
Environnement d'entraînement et d'évalutation
"""
# L'environnement python pour l'entraînement
train_py_env = suite_gym.load(env_name)
# L'environnement python pour l'évaluation
eval_py_env = suite_gym.load(env_name)
 
# On crée de nouveaux environnemens pour changer l'observation de ces derniers
t = NewEnv(train_py_env)
e = NewEnv(eval_py_env)

# L'environnement tensorflow pour l'entraînement
train_env = tf_py_environment.TFPyEnvironment(t)
# L'environnement tensorflow pour l'évaluation
eval_env = tf_py_environment.TFPyEnvironment(e)

"""
Réseau de neurones
"""
# Une première couche avec 120 neurones puis une seconde avec 60 neurones
fc_layer_params = (120, 60)
# On utilise pas de couches de convolution dans cet exemple car on observe des tableaux à une dimension

# Fonction Q
q_net = q_network.QNetwork(
    train_env.observation_spec(),
    train_env.action_spec(),
    fc_layer_params=fc_layer_params)

"""
Agent
"""
# Optimiseur avec en paramètre le taux d'apprentissage
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Compteur pour l'entraînement
train_step_counter = tf.Variable(0)

agent = dqn_agent.DqnAgent(
      # Spécificités d'un état
      train_env.time_step_spec(),
      # Spécifictés d'une action
      train_env.action_spec(),
      # Le réseau de neurones défini précédemment
      q_network=q_net,
      epsilon_greedy=epsilon_greedy,
      n_step_update=n_step_update,
      target_update_tau=target_update_tau,
      target_update_period=target_update_period,
      optimizer=optimizer,
      # une fonction pour calculer les TD errors loss
      td_errors_loss_fn=common.element_wise_squared_loss,
      gamma=gamma,
      reward_scale_factor=reward_scale_factor,
      train_step_counter=train_step_counter)

# On initialise notre agent
agent.initialize()

"""
Stratégies
"""
# La stratégie utilisée pour l'évaluation et le déploiement
eval_policy = agent.policy
# Une seconde stratégie pour la collecte de données
collect_policy = agent.collect_policy

# stratégie aléatoire
random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(),
                                                train_env.action_spec())

"""
Replay Buffer

Celui-ci garde une trace des données collectées
"""

# On lui donne en paramètre un tuple : 'Trajectory' qui contient les spécifications
# des obsevations, actions, récompenses et autres pour l'agent
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=train_env.batch_size,
    max_length=replay_buffer_max_length)

"""
On remplie le replay_buffer avec des actions choisies aléatoirement. Celui-ci ne doit pas être vide au début.
"""
# Le nombre de steps lors de l'initialisation est un hyperparamètre
collect_data(train_env, random_policy, replay_buffer, initial_collect_steps)

# Le replay buffer est donc maintenant une collection de 'Trajectoires'

"""
Dataset
"""
# L'agent a besoin d'accéder au replay buffer. On utilise alors ce dataset qui va 
# fournir les données à l'agent

# Dataset generates trajectories with shape [Bx2x...]
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3, 
    sample_batch_size=batch_size, 
    num_steps=2).prefetch(3)

iterator = iter(dataset)

###########################
# Entraînement de l'agent #
###########################

try:
  %%time
except:
  pass

# (Optional) Optimize by wrapping some of the code in a graph using TF function.
agent.train = common.function(agent.train)

# On reset le compteur pour l'entraînement
agent.train_step_counter.assign(0)

# On évalue la stratégie de l'agent avant l'entraînement
avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
returns = [avg_return]

# Le nombre d'iterations est un hyperparamètre de l'algorithme
for _ in range(num_iterations):

  # On collecte quelques steps en utilisant la stratégie de collecte de l'agent et on les sauvegarde dans le replay buffer
  collect_data(train_env, agent.collect_policy, replay_buffer, collect_steps_per_iteration)

  # Regroupement d'un échantillon de données du replay buffer et on met à jour le réseau de l'agent 
  experience, unused_info = next(iterator)
  train_loss = agent.train(experience).loss

  step = agent.train_step_counter.numpy()

  # on affiche ces informations en fonction des hyperparamètres définis précédemment
  if step % log_interval == 0:
    print('step = {0}: loss = {1}'.format(step, train_loss))

  if step % eval_interval == 0:
    avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
    print('step = {0}: Average Return = {1}'.format(step, avg_return))
    returns.append(avg_return)



In [ ]:
# Création d'une vidéo avec la stratégie définie en paramètre
def create_policy_eval_video(policy, filename, num_episodes=5, fps=30):
  filename = filename + ".mp4"
  with imageio.get_writer(filename, fps=fps) as video:
    for _ in range(num_episodes):
      time_step = eval_env.reset()
      video.append_data(eval_py_env.render())
      while not time_step.is_last():
        action_step = policy.action(time_step)
        time_step = eval_env.step(action_step.action)
        video.append_data(eval_py_env.render())
  return embed_mp4(filename)

In [ ]:
# Création d'une vidéo pour un agent entraîné où celui-ci va jouer 5 parties
create_policy_eval_video(agent.policy, "trained-agent")

In [ ]:
# Comparaison avec un agent qui utilise une stratégie aléatoire
create_policy_eval_video(random_policy, "random-agent")

Bibliographie : 

https://www.tensorflow.org/agents/tutorials/1_dqn_tutorial

https://pathtopioneer.com/blog/2020/07/rl-3#hyperparameters

